# Proof-of-Conecpt notepad

## fetching RSS feed by feedparser

In [12]:
import requests
import json
from datetime import datetime, timezone, timedelta
from bs4 import BeautifulSoup

from time import sleep
from random import randint
from collections import namedtuple

NOW: str = datetime.now(tz=timezone(timedelta(hours=9))).strftime("%Y%m%d%H%M")
sections = {
    "politics" : 100,
    "economy": 101,
    "social": 102,
    "life": 103,
    "world": 104,
    "science": 105 
}

Article = namedtuple("Article", ["title", "content", "url", "pub_time", "section", "press"])

articles: list[str | Article] = [NOW]

header: dict = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}

for section in sections.keys():
    for page in range(1):
        url = f"https://news.naver.com/section/template/SECTION_ARTICLE_LIST?sid={sections[section]}&sid2=&cluid=&pageNo={page}&date=&next={NOW}"
        response = requests.get(url, headers=header)
        bs = BeautifulSoup(json.loads(response.text)["renderedComponent"]["SECTION_ARTICLE_LIST"], 'html')
        sleep(0.5 + randint(0, 100) * 0.1)

        for element in bs.findAll("li"):
            url: str = element.select("a")[0]["href"].strip()
            response = requests.get(url, headers=header)

            articles.append(
                Article(
                    title=element.select("strong")[0].text.strip(),
                    # content=element.select(".sa_text_lede")[0].text.strip(),
                    content=BeautifulSoup(response.text).select("#newsct_article")[0].text.strip(),
                    url=element.select("a")[0]["href"].strip(),
                    pub_time=element.select(".sa_text_datetime")[0].text.strip(),
                    section=section,
                    press=element.select(".sa_text_press")[0].text.strip()
                )
            )

        sleep(0.5 + randint(0, 100) * 0.1)

    print(section)


politics


In [ ]:
articles

In [18]:
import csv
articles.pop(0)
articles.insert(0, tuple(title for title in Article._fields))
with open("./articles.csv", 'w', encoding="utf8") as f:
    csv.writer(f).writerows(articles)

## test LLM

In [ ]:
from llama_cpp import Llama
from transformers import AutoTokenizer

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='/home/gpp/src/model/llama3-korean-bllossom-8b/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf',
    n_ctx=4096,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

# PROMPT = \
# '''당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
# You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.'''

# instruction = '내일 개강을 맞이하는 대학생은 무엇을 해야 합니까?'

# messages = [
#     {"role": "system", "content": f"{PROMPT}"},
#     {"role": "user", "content": f"{instruction}"}
#     ]

# prompt = tokenizer.apply_chat_template(
#     messages, 
#     tokenize = False,
#     add_generation_prompt=True
# )

# generation_kwargs = {
#     "max_tokens":1024,
#     "stop":["<|eot_id|>"],
#     "top_p":0.9,
#     "temperature":0.6,
#     "echo":True, # Echo the prompt in the output
# }

# resonse_msg = model(prompt, **generation_kwargs)
# print(resonse_msg['choices'][0]['text'][len(prompt):])


In [ ]:
import requests

response = requests.get("https://feeds.feedburner.com/geeknews-feed")
rss_feed: str = response.content.decode()

print("RSS feed fetched successfully.")

rss_feed